
![alt text](edmond.png "generator discriminator formula")

# Programming Imagination : An introduction to Generative Adversarial Networks with Python

Christie’s auctioneers in New York City recently made international headlines for selling a portrait of Edmond de Belamy, who appears to be of the French Aristocracy, for a sum of $432,500. This isn't an unusual feat for an auction house that caters specifically to those of more discerning tastes. What makes this photo stand out isn't just the painting style or its masterful use of water colors; the artist that created this master piece is a neural network developed by a team in France. It's almost as if this network had an imagination of its own! That's what we'll dip into today, building imaginary features with Generative Adversarial Networks.

Generative Adversarial Networks (referred to as GAN(s) for brevity) are currently one of the hottest topics in machine learning, due to what they can accomplish with training data and what they can output. Rather than being a purely discriminatory network that can map labels onto features after it learns about the underlying patterns of that data, GAN's takes this in a backwards manner and can generate *possible* features after learning about the given underlying patterns.

## The method behind the madness of GANs

At a high level, this type of neural network architecture is all about generation of new data and validation of that newly generated data. Let's go over the parts of this system and their specific duties:

- The **generator network** takes the training data and based on what features it sees, generates new instances of that training data
    - If we were using the titanic data set it would create a whole new passenger that was never on the titanic
    - If we used the MNIST data set, it would generate new "numbers" that are not present in the current arabic numerals, butwould be close enough to pass
    - It then takes this newly created data along with samples from the actual data set and feeds them into the discriminator network

- The **discriminator network** a more traditional neural network that takes the outputs the generator network  has created and classifies said inputs with a probability of them being real or fake (this being a value between 0-1)

This is the equivalent of having an art critic and a counterfeit art maker in competition with eachother.
The counterfeit art maker (generator) initially only has accesss to the real paintings and attempts to make paintings are close to those as possible, with a few changes for originality (imagine someone "finding" a Rembrant that doesnt appear in his his known catalogue and attemping to put that to auction).

The art critic (the discriminator) appraises the paintings, both real and fake that are submitted to the auction house, and through careful study that the critic has built up over seeing many samples of work, attempts to make sure that fake images don't make it to the auction.

Over a series of training epochs, these two networks both get better at their respective jobs, maximizing their loss against eachother until a happpy medium is met.

The mathematical formula for this is the following:



![alt text](minmaxDG.png "generator discriminator formula")



D is the discrimator and G is the generator. This formula describes how the loss functions of each contribute to the training of the network

## Best Practices in handling dataand training GANs

Luckily for us using python, we have access to Kera's sequential models for training our neural networks. This is important becuase when training the both the generative and discrimanator network, these need to be done one after another, generally in the same function or part of script (It's good practice to see these  placed into a OO style class for simplicity, but those working in languages such as Scala might be used to a more functional approach). The reason for this is that each of the networks should be training against an already trained static adversary, by static we mean that the outputs and inputs that are being trained on are valid for the single for-loop epoch that both are being pitted agaisnt eachother.

The order of operations that has yeiled the best and most consistent results is to train the discriminator first, so the generative network can better capture the gradient that this discriminator uses to classify the results as fake or real.
This allows the generative network to better learn not just the data that is it going to generate better but also to "fool" the discriminator.

Just like any other neural network architecture, one must be careful of having learning rates that are too high or too low, but in this case this must be accounted for on both networks. Example, too high of a learning rate for the generative network and too low of a learning rate for the disciminator network would result in the generator far outclasssing the discriminator to the point of returning false negatives, as anything outside of what the discriminator "knows" based on training data will be labeled as fake. This is the equivalent of having an almost perfect counterfeit in the same style of a painter, but it having a single stroke out of place from what is deemed normal for that painter, and therefore being automatically labeled as a fake.


The following is a solid base example of GAN architecture taken from the github of Erik Linder-Norén, an up and coming machine learning engineer performing research out of Stocholm, Sweden. As discussed earlier, the usual program design is that of a python class that handles the training, building, and the execution of the sample images. 

In [4]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

95 [D loss: 0.016961, acc.: 100.00%] [G loss: 4.715963]
96 [D loss: 0.012682, acc.: 100.00%] [G loss: 4.738014]
97 [D loss: 0.009415, acc.: 100.00%] [G loss: 4.704195]
98 [D loss: 0.020551, acc.: 100.00%] [G loss: 4.449888]
99 [D loss: 0.011614, acc.: 100.00%] [G loss: 4.649545]
100 [D loss: 0.010379, acc.: 100.00%] [G loss: 4.599612]
101 [D loss: 0.015407, acc.: 100.00%] [G loss: 4.751950]
102 [D loss: 0.011038, acc.: 100.00%] [G loss: 4.637539]
103 [D loss: 0.008803, acc.: 100.00%] [G loss: 4.700588]
104 [D loss: 0.013487, acc.: 100.00%] [G loss: 4.792884]
105 [D loss: 0.017338, acc.: 100.00%] [G loss: 4.663293]
106 [D loss: 0.011979, acc.: 100.00%] [G loss: 4.654408]
107 [D loss: 0.014221, acc.: 100.00%] [G loss: 4.850651]
108 [D loss: 0.012273, acc.: 100.00%] [G loss: 4.753839]
109 [D loss: 0.019627, acc.: 100.00%] [G loss: 4.857789]
110 [D loss: 0.034477, acc.: 98.44%] [G loss: 4.807249]
111 [D loss: 0.015985, acc.: 100.00%] [G loss: 4.887391]
112 [D loss: 0.016295, acc.: 100.00%]

241 [D loss: 0.397935, acc.: 71.88%] [G loss: 2.573387]
242 [D loss: 0.728399, acc.: 53.12%] [G loss: 1.165167]
243 [D loss: 0.357296, acc.: 81.25%] [G loss: 2.405648]
244 [D loss: 0.575382, acc.: 71.88%] [G loss: 1.708871]
245 [D loss: 0.361432, acc.: 84.38%] [G loss: 2.441792]
246 [D loss: 0.625338, acc.: 64.06%] [G loss: 1.776830]
247 [D loss: 0.498867, acc.: 73.44%] [G loss: 2.485036]
248 [D loss: 0.973903, acc.: 40.62%] [G loss: 0.960581]
249 [D loss: 0.525656, acc.: 67.19%] [G loss: 2.537309]
250 [D loss: 0.762464, acc.: 46.88%] [G loss: 1.663648]
251 [D loss: 0.399343, acc.: 89.06%] [G loss: 1.855358]
252 [D loss: 0.653278, acc.: 60.94%] [G loss: 1.532038]
253 [D loss: 0.552832, acc.: 65.62%] [G loss: 1.980759]
254 [D loss: 0.926170, acc.: 32.81%] [G loss: 0.857503]
255 [D loss: 0.541738, acc.: 60.94%] [G loss: 1.775336]
256 [D loss: 0.901147, acc.: 42.19%] [G loss: 1.272664]
257 [D loss: 0.527284, acc.: 73.44%] [G loss: 1.511667]
258 [D loss: 0.739183, acc.: 54.69%] [G loss: 1.

390 [D loss: 0.674907, acc.: 48.44%] [G loss: 0.715188]
391 [D loss: 0.657225, acc.: 48.44%] [G loss: 0.698661]
392 [D loss: 0.701956, acc.: 46.88%] [G loss: 0.679703]
393 [D loss: 0.669658, acc.: 45.31%] [G loss: 0.697219]
394 [D loss: 0.665949, acc.: 51.56%] [G loss: 0.686090]
395 [D loss: 0.656953, acc.: 50.00%] [G loss: 0.687032]
396 [D loss: 0.672567, acc.: 51.56%] [G loss: 0.700465]
397 [D loss: 0.669778, acc.: 51.56%] [G loss: 0.694926]
398 [D loss: 0.660908, acc.: 50.00%] [G loss: 0.687938]
399 [D loss: 0.684723, acc.: 42.19%] [G loss: 0.670984]
400 [D loss: 0.702574, acc.: 51.56%] [G loss: 0.673622]
401 [D loss: 0.658703, acc.: 46.88%] [G loss: 0.694423]
402 [D loss: 0.679987, acc.: 50.00%] [G loss: 0.704944]
403 [D loss: 0.675825, acc.: 45.31%] [G loss: 0.689096]
404 [D loss: 0.685418, acc.: 42.19%] [G loss: 0.702914]
405 [D loss: 0.688924, acc.: 51.56%] [G loss: 0.689394]
406 [D loss: 0.675594, acc.: 50.00%] [G loss: 0.711905]
407 [D loss: 0.688147, acc.: 48.44%] [G loss: 0.

538 [D loss: 0.663768, acc.: 56.25%] [G loss: 0.725203]
539 [D loss: 0.678380, acc.: 54.69%] [G loss: 0.755716]
540 [D loss: 0.665436, acc.: 56.25%] [G loss: 0.750229]
541 [D loss: 0.693199, acc.: 48.44%] [G loss: 0.765527]
542 [D loss: 0.668285, acc.: 54.69%] [G loss: 0.760979]
543 [D loss: 0.666310, acc.: 56.25%] [G loss: 0.761629]
544 [D loss: 0.699820, acc.: 48.44%] [G loss: 0.723093]
545 [D loss: 0.653743, acc.: 57.81%] [G loss: 0.745459]
546 [D loss: 0.671442, acc.: 51.56%] [G loss: 0.725014]
547 [D loss: 0.655211, acc.: 56.25%] [G loss: 0.749019]
548 [D loss: 0.647312, acc.: 59.38%] [G loss: 0.769560]
549 [D loss: 0.659388, acc.: 51.56%] [G loss: 0.764105]
550 [D loss: 0.661525, acc.: 62.50%] [G loss: 0.735016]
551 [D loss: 0.634802, acc.: 59.38%] [G loss: 0.752713]
552 [D loss: 0.617151, acc.: 65.62%] [G loss: 0.773436]
553 [D loss: 0.640293, acc.: 62.50%] [G loss: 0.768444]
554 [D loss: 0.651517, acc.: 57.81%] [G loss: 0.768964]
555 [D loss: 0.647143, acc.: 60.94%] [G loss: 0.

...

29343 [D loss: 0.594288, acc.: 65.62%] [G loss: 1.139203]
29344 [D loss: 0.576782, acc.: 62.50%] [G loss: 1.010771]
29345 [D loss: 0.652992, acc.: 64.06%] [G loss: 1.093743]
29346 [D loss: 0.664225, acc.: 62.50%] [G loss: 1.014993]
29347 [D loss: 0.601725, acc.: 70.31%] [G loss: 0.942583]
29348 [D loss: 0.580321, acc.: 73.44%] [G loss: 0.954350]
29349 [D loss: 0.566474, acc.: 68.75%] [G loss: 1.112961]
29350 [D loss: 0.755453, acc.: 46.88%] [G loss: 1.035961]
29351 [D loss: 0.584218, acc.: 71.88%] [G loss: 0.975796]
29352 [D loss: 0.603667, acc.: 70.31%] [G loss: 1.055803]
29353 [D loss: 0.618191, acc.: 67.19%] [G loss: 1.176837]
29354 [D loss: 0.539594, acc.: 73.44%] [G loss: 1.038338]
29355 [D loss: 0.559510, acc.: 67.19%] [G loss: 1.079043]
29356 [D loss: 0.652852, acc.: 64.06%] [G loss: 1.036737]
29357 [D loss: 0.675793, acc.: 64.06%] [G loss: 1.045763]
29358 [D loss: 0.560644, acc.: 70.31%] [G loss: 1.046502]
29359 [D loss: 0.714459, acc.: 48.44%] [G loss: 1.085910]
29360 [D loss:

29485 [D loss: 0.575946, acc.: 70.31%] [G loss: 1.022702]
29486 [D loss: 0.609446, acc.: 64.06%] [G loss: 1.127104]
29487 [D loss: 0.625417, acc.: 68.75%] [G loss: 1.037825]
29488 [D loss: 0.603212, acc.: 68.75%] [G loss: 0.913630]
29489 [D loss: 0.614249, acc.: 68.75%] [G loss: 1.063043]
29490 [D loss: 0.689616, acc.: 51.56%] [G loss: 0.966537]
29491 [D loss: 0.569034, acc.: 73.44%] [G loss: 1.137352]
29492 [D loss: 0.567435, acc.: 75.00%] [G loss: 1.138528]
29493 [D loss: 0.679863, acc.: 65.62%] [G loss: 1.031180]
29494 [D loss: 0.681353, acc.: 56.25%] [G loss: 1.037647]
29495 [D loss: 0.595095, acc.: 71.88%] [G loss: 1.065524]
29496 [D loss: 0.682509, acc.: 46.88%] [G loss: 1.086385]
29497 [D loss: 0.593466, acc.: 57.81%] [G loss: 0.885156]
29498 [D loss: 0.587532, acc.: 70.31%] [G loss: 1.040096]
29499 [D loss: 0.605608, acc.: 65.62%] [G loss: 0.992804]
29500 [D loss: 0.554097, acc.: 73.44%] [G loss: 1.085252]
29501 [D loss: 0.631033, acc.: 64.06%] [G loss: 0.926501]
29502 [D loss:

29630 [D loss: 0.643924, acc.: 68.75%] [G loss: 1.065437]
29631 [D loss: 0.589629, acc.: 62.50%] [G loss: 1.078011]
29632 [D loss: 0.581492, acc.: 64.06%] [G loss: 1.026736]
29633 [D loss: 0.660289, acc.: 62.50%] [G loss: 1.026918]
29634 [D loss: 0.650310, acc.: 60.94%] [G loss: 1.248140]
29635 [D loss: 0.593662, acc.: 75.00%] [G loss: 1.022778]
29636 [D loss: 0.532288, acc.: 67.19%] [G loss: 1.039016]
29637 [D loss: 0.659319, acc.: 64.06%] [G loss: 0.984553]
29638 [D loss: 0.719589, acc.: 53.12%] [G loss: 0.926293]
29639 [D loss: 0.599297, acc.: 70.31%] [G loss: 0.978809]
29640 [D loss: 0.606078, acc.: 68.75%] [G loss: 0.998600]
29641 [D loss: 0.593937, acc.: 68.75%] [G loss: 1.151420]
29642 [D loss: 0.666716, acc.: 54.69%] [G loss: 1.026298]
29643 [D loss: 0.589196, acc.: 71.88%] [G loss: 0.962436]
29644 [D loss: 0.650521, acc.: 59.38%] [G loss: 1.002962]
29645 [D loss: 0.640078, acc.: 57.81%] [G loss: 0.915543]
29646 [D loss: 0.612836, acc.: 71.88%] [G loss: 0.944904]
29647 [D loss:

29774 [D loss: 0.655957, acc.: 60.94%] [G loss: 1.038785]
29775 [D loss: 0.673411, acc.: 53.12%] [G loss: 1.052261]
29776 [D loss: 0.636298, acc.: 60.94%] [G loss: 1.183244]
29777 [D loss: 0.611307, acc.: 60.94%] [G loss: 1.223676]
29778 [D loss: 0.587313, acc.: 70.31%] [G loss: 1.075671]
29779 [D loss: 0.619689, acc.: 67.19%] [G loss: 1.059842]
29780 [D loss: 0.728750, acc.: 54.69%] [G loss: 0.973502]
29781 [D loss: 0.575448, acc.: 64.06%] [G loss: 1.008412]
29782 [D loss: 0.626023, acc.: 70.31%] [G loss: 0.975975]
29783 [D loss: 0.635399, acc.: 67.19%] [G loss: 1.144824]
29784 [D loss: 0.695273, acc.: 56.25%] [G loss: 0.979464]
29785 [D loss: 0.640430, acc.: 67.19%] [G loss: 0.934434]
29786 [D loss: 0.659832, acc.: 60.94%] [G loss: 1.018318]
29787 [D loss: 0.615063, acc.: 68.75%] [G loss: 1.034956]
29788 [D loss: 0.616169, acc.: 62.50%] [G loss: 0.915669]
29789 [D loss: 0.604770, acc.: 67.19%] [G loss: 1.040564]
29790 [D loss: 0.541921, acc.: 79.69%] [G loss: 1.115284]
29791 [D loss:

29917 [D loss: 0.588969, acc.: 71.88%] [G loss: 0.955779]
29918 [D loss: 0.630540, acc.: 67.19%] [G loss: 1.065346]
29919 [D loss: 0.669220, acc.: 62.50%] [G loss: 0.991709]
29920 [D loss: 0.693051, acc.: 64.06%] [G loss: 1.023222]
29921 [D loss: 0.596865, acc.: 67.19%] [G loss: 0.880121]
29922 [D loss: 0.647040, acc.: 68.75%] [G loss: 1.022046]
29923 [D loss: 0.553124, acc.: 71.88%] [G loss: 1.143832]
29924 [D loss: 0.597866, acc.: 67.19%] [G loss: 0.997138]
29925 [D loss: 0.573446, acc.: 70.31%] [G loss: 1.137020]
29926 [D loss: 0.577387, acc.: 67.19%] [G loss: 1.144742]
29927 [D loss: 0.640473, acc.: 64.06%] [G loss: 1.108928]
29928 [D loss: 0.654199, acc.: 64.06%] [G loss: 1.092082]
29929 [D loss: 0.530789, acc.: 67.19%] [G loss: 1.034080]
29930 [D loss: 0.600798, acc.: 68.75%] [G loss: 1.063042]
29931 [D loss: 0.543251, acc.: 71.88%] [G loss: 1.046154]
29932 [D loss: 0.536593, acc.: 73.44%] [G loss: 1.113661]
29933 [D loss: 0.600703, acc.: 68.75%] [G loss: 1.060639]
29934 [D loss:

## Uses of Gans in production

GANs have made large headways not just in image generation, a few of these are:
- Partial image reconstruction
- 3D space generation
- Image to image translation
- Mapping clothes onto models/people for online shopping stores
- Text generation for NLP
- Makeup-Invariant Face Verification
